In [1]:
import sys
import os

# Add project root to Python path
project_root = os.path.abspath('.')
sys.path.append(project_root)

# Add Related_Reps directory to Python path
related_reps_path = os.path.join(project_root, 'Related_Reps')
sys.path.append(related_reps_path)

# Add RepBublik directory to Python path
repbublik_path = os.path.join(related_reps_path, 'RePBubLik', 'RepBublik')
sys.path.append(repbublik_path)

%load_ext autoreload
%autoreload 2
from src.mitigation import MitigationComparison, MitigationStrategy, MitigationMeasurement
from src.opinion_dynamics import OpinionDynamics


In [11]:
strategies = ['A_CD', 'A_RL+', 'A_ROV', 'AW_P', 'R_WCR', 'R_O']
# strategies = ['AW_P']
graph_file = 'output/results-theta=0.5/Referendum_/graph.gml'
MC = MitigationComparison(graph_file=graph_file, num_edges=1000, strategies=strategies)
perc_rb = 0.1
ratio = 100

Graph is not connected, adding edges to connect the components
The graph is connected now


In [12]:
result = MC.generate_strategies(perc_rb=perc_rb, ratio=ratio)

Adding A_CD with 1000 edges
Adding A_RL+ with 1000 edges
Adding A_ROV with 1000 edges
Adding AW_P with 1000 edges
Reweighting R_WCR with 1000 edges
Reweighting R_O with 1000 edges
Adding AW_MU with 1000 edges
Adding AW_MW with 1000 edges
Adding AW_G with 1000 edges


In [13]:
ops = {}
metrics = {}

for s in strategies:
    print('Processing: ', s)
    G = MC.get_graph().copy()
    new_edges = []
    new_weights = {}
    if s[0] == 'R':
        new_weights = result[s]['new_weights']
        new_edges = list(new_weights.keys())
    elif s[0] == 'A':
        new_edges = result[s]['new_edges']
        # convert items in new_edges to tuple
        new_edges = [tuple(edge) for edge in new_edges]
        new_weights = {edge: 1 for edge in new_edges}

    # Create a new graph with the same nodes as the original graph
    op_dynamics = OpinionDynamics(G)
    ops[s] = op_dynamics.run(model='friedkin_johnsen_simplified_matrix', add_connections=new_edges, reweight=new_weights)

    MM_s = MitigationMeasurement(G)
    G.add_edges_from(new_edges)
    MM_t = MitigationMeasurement(G, ops[s][-1])

    metrics[s] = {}
    print('Computing variance for ', s)
    metrics[s]['var'] = [MM_s.opinion_variance(), MM_t.opinion_variance()]
    print('Computing controversy for ', s)
    metrics[s]['controversy'] = [MM_s.opinion_controversy(), MM_t.opinion_controversy()]
    print('Computing rwc for ', s)
    # Note: cannot use MM_t for rwc, as it will label red and blue nodes based on the final state
    metrics[s]['rwc'] = [MM_s.rwc(perc_rb=perc_rb), MM_s.rwc(perc_rb=perc_rb, add_edges=new_edges)]
    print('Computing ged for ', s)
    # Note: for ged, we don't apply the opinion changes, only quantify the new_edges' effect
    MM_t = MitigationMeasurement(G)
    metrics[s]['ged'] = [MM_s.ged(), MM_t.ged()]

Processing:  A_CD
Computing variance for  A_CD
Computing controversy for  A_CD
Computing rwc for  A_CD
Computing ged for  A_CD
Processing:  A_RL+
Computing variance for  A_RL+
Computing controversy for  A_RL+
Computing rwc for  A_RL+
Computing ged for  A_RL+
Processing:  A_ROV
Computing variance for  A_ROV
Computing controversy for  A_ROV
Computing rwc for  A_ROV
Computing ged for  A_ROV
Processing:  AW_P
Computing variance for  AW_P
Computing controversy for  AW_P
Computing rwc for  AW_P
Computing ged for  AW_P
Processing:  R_WCR
Computing variance for  R_WCR
Computing controversy for  R_WCR
Computing rwc for  R_WCR
Computing ged for  R_WCR
Processing:  R_O
Computing variance for  R_O
Computing controversy for  R_O
Computing rwc for  R_O
Computing ged for  R_O
Processing:  AW_MU
Computing variance for  AW_MU
Computing controversy for  AW_MU
Computing rwc for  AW_MU
Computing ged for  AW_MU
Processing:  AW_MW
Computing variance for  AW_MW
Computing controversy for  AW_MW
Computing rwc 

In [14]:
import pandas as pd

# Create a DataFrame from the metrics dictionary
df = pd.DataFrame(metrics)
# Convert all values in the DataFrame to round(3) floats
# if it is a list, convert items in it to round(3) values
for col in df.columns:
    if df[col].apply(lambda x: isinstance(x, list)).any():
        df[col] = df[col].apply(lambda x: [round(item, 3) for item in x])

# Create a table with the metrics
table = df.to_markdown()
print(table)

|             | A_CD            | A_RL+           | A_ROV           | AW_P            | R_WCR           | R_O              | AW_MU           | AW_MW          | AW_G           |
|:------------|:----------------|:----------------|:----------------|:----------------|:----------------|:-----------------|:----------------|:---------------|:---------------|
| var         | [0.469, 0.043]  | [0.469, 0.214]  | [0.469, 0.091]  | [0.469, 0.079]  | [0.469, 0.227]  | [0.469, 0.271]   | [0.469, 0.081]  | [0.469, 0.079] | [0.469, 0.088] |
| controversy | [0.85, 0.425]   | [0.85, 0.595]   | [0.85, 0.472]   | [0.85, 0.46]    | [0.85, 0.608]   | [0.85, 0.652]    | [0.85, 0.462]   | [0.85, 0.46]   | [0.85, 0.469]  |
| rwc         | [0.59, 0.324]   | [0.59, 0.525]   | [0.59, 0.368]   | [0.59, 0.338]   | [0.59, 0.546]   | [0.59, 0.59]     | [0.59, 0.343]   | [0.59, 0.337]  | [0.59, 0.356]  |
| ged         | [10.012, 6.394] | [10.012, 9.137] | [10.012, 7.493] | [10.012, 7.212] | [10.012, 8.891] | [10.012, 

In [15]:
# # for all s in instrategies, there are a list of two dicts in ops[s]
# # each dict is a map from node id to opinion value
# # dict 0 is for the initial state, dict 1 is for the final state
# # draw a heatmap for the change of opinion values between the two dicts
# # x-axis as the initial opinion, y-axis as the final opinion
# # the value of the heatmap is the count of the opinion changes from x to y
# # use log scale for the value of the heatmap

# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.colors import LogNorm

# # Create a figure with subplots for each strategy
# fig, axes = plt.subplots(2, 3, figsize=(12, 6))
# axes = axes.flatten()

# # For each strategy
# for idx, s in enumerate(strategies):
#     # Get initial and final opinions
#     initial_ops = list(ops[s][0].values())
#     final_ops = list(ops[s][1].values())
    
#     # Create bins for the heatmap
#     bins = np.linspace(-1, 1, 21)  # 20 bins from -1 to 1
    
#     # Create 2D histogram
#     H, xedges, yedges = np.histogram2d(initial_ops, final_ops, bins=[bins, bins])
    
#     # Plot heatmap
#     im = axes[idx].imshow(H.T, origin='lower', extent=[-1, 1, -1, 1],
#                          aspect='auto', cmap='viridis', norm=LogNorm())
    
#     # Add colorbar
#     plt.colorbar(im, ax=axes[idx])
    
#     # Add labels and title
#     axes[idx].set_xlabel('Initial Opinion')
#     axes[idx].set_ylabel('Final Opinion')
#     axes[idx].set_title(s)
    
#     # Add diagonal line
#     axes[idx].plot([-1, 1], [-1, 1], 'r--', alpha=0.5)

# plt.tight_layout()
# plt.show()